<div class="alert alert-danger">
    <b>ACHTUNG</b>: Das Kapitel befindet sich noch in Arbeit und ist nicht fertig.
</div>

# Zusatzkapitel 2 - Neuronale Netze mit PyTorch und TorchText

## Z.2.1. Kapitelübersicht <a class="anchor" id="Z-2-1"/>

In diesem Kapitel wird gezeigt, wie man einfache Neuronale Netze mit der Deep-Learning-Bibliothek **PyTorch** und **TorchText** erstellt. Dieses Kapitel baut auf den Kapiteln 10 bis 13 auf, d.h. das Konzept von Neuronalen Netzen, gängige Deep Learning Begriffe und erste Praxiserfahrungen mit Keras sollten bekannt und vorhanden sein. Weiterhin sollte die Benutzung von Iteratoren in Python sowie das Konzept der objektorientierten Programmierung in Python geläufig sein, da PyTorch eine Klassenarchitektur zur Erstellung von Modellen benutzt. Auch wird die Benutzung einer GPU wie bei Kapitel 12 dringend empfohlen (auch hier kann wieder **Google Colab** benutzt werden).

TODO: Word Embeddings als Pflicht?

**Hinweis**: Das PyTorch-Modul befindet sich <u>nicht</u> in der bei dieser Tutorialreihe beigelegten `requirements.txt` Datei. Es muss eigenständig installiert werden, was jedoch im Gegensatz zu Keras oder Tensorflow weitaus unkomplizierter ist. Die Webseite von <a href="https://pytorch.org/">PyTorch</a> ist hier sehr hilfreich. <a href="https://github.com/pytorch/text">TorchText</a> muss ebenfalls noch installiert werden.

<b>Abschnittsübersicht</b>:<br>
[Z.2.1. Kapitelübersicht](#Z-2-1)<br>
[Z.2.2. Übersicht zu PyTorch](#Z-2-2)<br>
[Z.2.3. Vorverarbeitung des Korpus mit TorchText](#Z-2-3)<br>
[Z.2.3.1. Aufteilung des Korpus](#Z-2-3-1)<br>
[Z.2.3.2. Laden der Train-Val-Test-Dateien](#Z-2-3-2)<br>
[Z.2.3.3. Erstellung eines einfachen sequentiellen Modells TODO](#Z-2-4)<br>
[Z.2.3.2. Laden der JSON-Dateien](#Z-2-3-2)<br>
[Z.2.3.3. Erstellung eines sequentiellen Modells](#Z-2-3-3)<br>
[Z.2.3.4. Training eines sequentiellen Modells](#Z-2-3-4)<br>

Am Ende dieses Kapitel werden wir folgende Themen behandelt und/oder vertieft haben:
- Implementierung eines Neuronalen Netz mit PyTorch

In [13]:
from nltk import word_tokenize
from nltk.corpus import stopwords
punctuation = ['!', '#','$','%','&', "'", '(',')','*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', 
               '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '`', '``', 'wurde', 'wurden']
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
from torchtext import data, datasets

## Z.2.2. Übersicht zu PyTorch <a class="anchor" id="Z-2-2"/>

<div class="alert alert-warning">
<b>Aufgabe:</b> Deep Learning with PyTorch <br>
    
Für das folgende Tutorial ist es hilfreich, begleitend das Buch von Eli Stevens et. al. "Deep Learning with PyTorch" zu lesen, welches einen guten Einstieg zu PyTorch liefert. 
    
<br><u>Das Buch</u>: STEVENS, Eli et. al., Deep Learning with PyTorch, July 2020 (englische Ausgabe).
<br><br>
    
Es eignet sich sehr gut für Leser, die bereits ersten Einblicke im Deep Learning gewonnen haben, wie etwa durch das Buch von Chollet (siehe Kapitel 10-12). Anders als bei dem Buch von Chollet wird das Buch von Stevens et. al. nicht für die Benutzung dieses Tutorials vorausgesetzt, es dient eher als Begleitlektüre mit vertiefenden, erweiterenden und ausführlicheren Behandlung von Themen. Die Autoren scheinen bei den Themen einen Fokus auf die Arbeit mit Bilddateien zu legen, weshalb ich im folgenden Kapitel hervorheben möchte, die für die Arbeit mit Textdaten hilfreich sind oder diese als Thema haben. Sollte man das Buch also nur überfliegen, sollten diese Kapitel mindestens oder ausführlicher gelesen werden:
- Kapitel 3: "It starts with a tensor". Gute Einführung zum Datentyp <b>Tensor</b>, der für PyTorch eine wichtige Rolle spielt.
- Kapitel 4.5: "Representing text". Zeigt, wie Textdaten mit PyTorch verarbeitet werden können und greift auch das Thema <b>Embeddings</b> auf.
- Kapitel 5: "The mechanics of learning". Hier werden noch einmal die Grundlagen von Neuronalen Netzen erklärt, aber auch einige PyTorch Eigenheiten, die wichtig sind.
- TODO
- ...

</div>

Wie auch **Keras** ist **PyTorch** eine Deep-Learning-Bibliothek, anders als Keras basiert PyTorch jedoch nicht auf der Bibliothek **Tensorflow**, sondern auf der Bibliothek **Torch**. PyTorch erfordert zudem eine stärkere Auseinandersetzung mit den Mechaniken von Deep Learning, weshalb auch mehr Code als bei Keras benötigt wird, in welchem meist mit nur wenigen Zeilen Code ein Neuronales Netz erstellt werden kann. Im akademischen Bereich wird PyTorch aktuell (Stand August 2020) jedoch viel häufiger als Keras verwendet, da es eine bessere Performance liefert und die Erstellung von komplexeren Neuronalen Netzwerkarchitekturen ermöglicht. Keras selbst wird anders als Tensorflow oder PyTorch aufgrund seiner Simplizität meistens nur zur Erstellung von Prototypen genutzt, abseits von Tutorials ist es deshalb seltener zu finden.

### PyTorch vs. Keras


<table align="left"> 
  <tr>
    <th></th>
    <th style="text-align:center;">Keras</th>
    <th style="text-align:center;">PyTorch</th>
  </tr>
  <tr>
      <td><b>Vorteile</b></td>
    <td>
        <ul style="text-align:left;">
          <li>Einsteigerfreundlich</li>
          <li>Simple Architektur</li>
          <li>Einfache Erstellung von Prototypen</li>
          <li>Gut für kleine Datensätze</li>
          <li>wenige Zeilen Code</li>
          <li>Debugging kaum notwendig</li>
        </ul> 
    </td>
    <td>
        <ul style="text-align:left;">
          <li>Sehr flexibel</li>
            <li>Höhere Perfomance</li>
            <li>Ermöglicht komplexere Neuronale Netzwerke</li>
            <li>Nutzt Python-Syntax</li>
            <li>Eignet sich gut fürs Debugging</li>
            <li>Gute Peformance auch für größere Datensätze</li>
            <li>Großer Community-Support</li>
            <li>Forschung nutzt vorrangig PyTorch, d.h. Paper nutzen PyTorch-Architektur</li>
        </ul>   
    </td>
  </tr>
  <tr>
    <td><b>Nachteile</b></td>
    <td>
        <ul style="text-align:left;">
          <li>Langsamer als PyTorch</li>
          <li>Benutzung der eigenen GPU kann umständlich werden</li>
            <li><i>Subjektiv:</i> Abgewandelte Python-Syntax</li>
            <li>Weniger Community-Support</li>
        </ul> 
    </td>
    <td>
        <ul style="text-align:left;">
          <li>Weniger einsteigerfreundlich als Keras</li>
          <li>Braucht viel mehr Code</li>
        </ul>    
    </td>
  </tr>
</table> 

Zu Beginn ist es recht nützlich, sich einen kurzen Überblick über die wichtigsten Module von PyTorch zu verschaffen, die im folgenden aufgelistet sind:

- `torch.nn`: Die Kern-Module, die zur Erstellung eines Neuronalen Netzes benötigt werden, befinden sich in diesem übergeordneten Modul. Vorwiegend befinden sich hier Module zur Erstellung von unterschieden **Layer** eines NN (z.B. fully connected Layer oder Convolutional Layer), aber auch **Aktivierungsfunktionen** und **Verlustfunktionen** können hiermit aufgerufen werden.
- `torch.optim`: Hier befinden sich Module zur Implementierung von **Optimierungsverfahren**. In Kapitel 12.5 wurden die Optimierungsverfahren *Adam*, *RMSprop* und *SGD* zur Hyperparameteroptimierung verwendet, diese stehen bei PyTorch ebenfalls zur Verfügung und können z.B. mit `from torch.optim as RMSprop` aufgerufen werden.
- `torch.tensor`: Dieses Modul befasst sich mit der Datenstruktur **Tensor**.
- `torch.utils.data`: Dieses Modul enthält die Klasse **`DataLoader`**, mithilfe derer Datensätze für die Verwendung von PyTorch geladen werden können.





TODO:<br>
- grund dinge wie torch modul & ähnliches erklären
- tabelle unten überpfüen und ggbf ergänzen



Sollte man mit Textdaten arbeiten, empfiehlt es sich, neben PyTorch noch die ebenfalls von PyTorch bereitgestelle Bibliothek **`TorchText`** zu verwenden. Diese vereinfacht typische Vorverarbeitungsschritte für die Arbeit mit Textdaten ungemein. Das Pendant für Bilddaten heißt **`TorchVision`** und das Pendant für Audiodaten heißt **`TorchAudio`**, beide Bibliotheken werden in diesem Tutorial jedoch nicht behandelt.

## Z.2.3. Vorverarbeitung des Korpus mit TorchText <a class="anchor" id="Z-2-3"/>

Das Einlesen und die Aufteilung des Korpus unterscheidet sich von dem Vorgehen in den anderen Kapiteln. PyTorch erfordert mehr noch als Keras eine ausführlichere Vorverarbeitung des Korpus. Für das Einlesen und für die Vorverarbeitung von Textdaten ist es daher sinnvoll, die Bibliothek **`TorchText`** zu verwenden. `TorchText` hilft bei den folgenden Operationen[<sup>1</sup>](#fn1):

- **Laden von Dateien**: Laden in das Korpus aus verschiedenen Formaten
- **Tokenisierung**: Sätze in Wortlisten aufteilen
- **Vokabular erstellen**: Erstellen einer Vokabelliste
- **Numerisieren/Indexifizieren**: Wörter eines Korpus als Ganzzahlen darstellen
- **Wortvektoren**: entweder das Vokabular zufällig initialisieren oder aus einem vortrainierten Embedding laden (dieses muss jedoch "getrimmt" werden, d.h. speichern nur Wörter aus den Vokabular werden auch gespeichert).
- **Batching**: Erzeugen von Batches von Trainingsdaten


Leider übernimmt TorchText nicht die folgenden Aufgaben, die manuell oder mit anderen Hilfsbibliotheken durchgeführt werden müssen:

- **Train-Val-Test-Split**: Korpus in Trainings-, Validierungs und Testdaten aufteilen
- **Embedding Lookup**: Zuordnung jedes Satzes (der Wortindizes enthält) zu Wortvektoren einer festen Dimension, welches für Word Embeddings relevant ist


<hr style="border: 0.1px solid black;"/>
<div id="fn1" style="font-size:8pt; line-height:1; padding-left: 1em; text-indent: -1em"><sup style="font-size:5pt">1</sup>&nbsp; Die Liste wurde von diesem <a href="https://anie.me/On-Torchtext/"> Blogpost</a> entnommen.</div>import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
from torchtext import data, datasets